Если база данных уже находится в папке то можно сразу запускать сайт на Flask c помощью файла flask_run.py

In [ ]:
!python flask_run.py

Стандартно сайт будет доступен по адресу http://127.0.0.1:5000/ или http://localhost:5000/  
Иначе проделать действия ниже

In [22]:
import pandas as pd
import numpy as np
import flask
#import sqlalchemy 
from sqlalchemy import create_engine, MetaData
from geolite2 import geolite2

### Этап загрузки логов


In [2]:
data_logs = pd.read_csv('data/logs.txt', sep=' ', header=None)  
data_logs = data_logs.drop([0,1,2,3,4,5,6,9,10], axis=1)
data_logs.columns = ['date','time','ip_addres','url']
data_logs["country"] = "empty"
i = 0
for val_data, val_time in zip(data_logs["date"], data_logs["time"]):
    data_logs["date"][i] = val_data +' '+ val_time
    reader = geolite2.reader()
    country = reader.get(data_logs["ip_addres"][i])
    if country is None:
        data_logs.country[i] = 'Narnia'
    else:
        try:
            country = country["country"]["names"]['en']
        except KeyError:
            country = country["registered_country"]["names"]['en']
        data_logs.country[i] = country    
    i += 1
data_logs =  data_logs.drop(["time"], axis=1)

In [3]:
data_logs.head(5);

### Запускаем парсер который подготовит логи к загрузке в базу

In [4]:
from parser_logs import run_parser
goods_category_df, goods_df, cart_df, pay_df = run_parser(data_logs)

C:\Users\Aleksey\my_first_project\parser_logs.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  goods_df.goods_id[j] = i + 1
C:\Users\Aleksey\my_first_project\parser_logs.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  goods_df.category_id[i] = j + 1


### Создаем базу данных

In [5]:
from create_db import Create_DataBase
engine = create_engine("sqlite:///data/DB_all_to_the_bottom.db")
metadata = MetaData()
metadata = Create_DataBase()

### Загружаем данные в базу

In [6]:
data_logs.to_sql('logs', con=engine, if_exists='append', index_label='logs_index')

category_and_goods = goods_df[["category_id","goods_id"]]
category_and_goods.to_sql('goods_and_category', con=engine, if_exists='append', index_label="goods_and_category_index")

goods = goods_df[["goods_id","name_goods"]]
goods.to_sql('goods', con=engine, if_exists='append', index_label="goods_index")

goods_category_df["category_id"] = range(1,goods_category_df.shape[0]+1)
goods_category_df = goods_category_df[["category_id","name_category"]]
goods_category_df.to_sql('goods_category', con=engine, if_exists='append', index_label="goods_category_index")

cart_df.to_sql('cart', con=engine, if_exists='append', index_label="cart_index")

pay_df.to_sql('pay', con=engine, if_exists='append', index_label="pay_index") 

In [ ]:
!python flask_run.py